In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""#token
!pip install huggingface_hub


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from huggingface_hub import login
login(token = os.environ["HUGGINGFACEHUB_API_TOKEN"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/jovyan/.cache/huggingface/token
Login successful


In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

In [1]:
model="meta-llama/Llama-2-13b-hf"

In [6]:
tokenizer=AutoTokenizer.from_pretrained(model)


In [7]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=4096,
    do_sample=True,
    top_k=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

2023-11-17 09:26:10.727234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 09:26:10.854068: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-17 09:26:11.476965: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-17 09:26:11.477050

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.5})

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

structure_prompt_template_text = """
Based on these examples, generate a structured script for a presentation on "{topic}" in about 10 slides. Ensure that each slide in the script:

Opens with a clear statement that introduces the distinct aspect of the topic.
Provides detailed points that give a comprehensive understanding of the slide's content, ideal for spoken narration.
Explores a different dimension of the topic to offer a well-rounded perspective.
Remember, the format should be consistent across all slides, with clear opening statements and detailed points, making the script easy to follow and effective for delivery.
Example presentation on Climate Change:
Topic: Climate Change

Slide 1: "Introduction to Climate Change"
Script: "Climate change is a significant global phenomenon characterized by long-term shifts in temperature and weather patterns, primarily due to human activities."

Slide 2: "Historical Perspective"
Script: "The history of climate change reveals a growing awareness and understanding, marked by scientific discoveries and public discourse from the late 19th century to the present."

Slide 3: "Causes of Climate Change"
Script: "Climate change results from natural processes and human-driven activities, with key contributors including greenhouse gas emissions, deforestation, and industrial practices."

Slide 4: "Effects on Global Weather"
Script: "Climate change is causing more frequent and severe weather events like hurricanes, droughts, and heatwaves, impacting ecosystems and human societies worldwide."

Slide 5: "Impact on Biodiversity"
Script: "The changing climate is leading to habitat destruction, species extinction, and altered ecosystems, posing threats to global biodiversity."

Slide 6: "Human Impact and Responses"
Script: "Human activities, notably fossil fuel combustion and land use changes, are primary drivers of climate change, prompting responses such as renewable energy development and conservation efforts."

Slide 7: "Policy and Regulation"
Script: "Governments and international organizations are formulating policies and regulations aimed at mitigating climate change, including international agreements like the Paris Accord."

Slide 8: "Technological Innovations"
Script: "Technological advancements play a crucial role in combating climate change, from renewable energy technologies to carbon capture and storage systems."

Slide 9: "Community and Cultural Impact"
Script: "Communities and cultures worldwide are adapting to climate change, incorporating sustainable practices and developing resilience against climate-related challenges."

Slide 10: "Future Perspectives"
Script: "Future projections of climate change indicate continuing temperature rises, sea-level rise, and an increase in extreme weather events, necessitating urgent and comprehensive action."

Answer presentation script on {topic}:
"""

structure_prompt_template_text = """
Based on these examples, generate a structured script for a presentation on "{topic}" in about 10 slides. Ensure that each slide in the script:

Opens with a clear statement that introduces the distinct aspect of the topic.
Provides detailed points that give a comprehensive understanding of the slide's content, ideal for spoken narration.
Explores a different dimension of the topic to offer a well-rounded perspective.
Remember, the format should be consistent across all slides, with clear opening statements and detailed points, making the script easy to follow and effective for delivery.
Example presentation on Climate Change:

Example presentation on Climate Change:
Topic: Climate Change

Slide 1: "Introduction to Climate Change"
Script: "Climate change is a significant global phenomenon characterized by long-term shifts in temperature and weather patterns, primarily due to human activities."
Illustration: "Earth half green, half industrial."

Slide 2: "Historical Perspective"
Script: "The history of climate change reveals a growing awareness and understanding, marked by scientific discoveries and public discourse from the late 19th century to the present."
Illustration: "Simple timeline."

Slide 3: "Causes of Climate Change"
Script: "Climate change results from natural processes and human-driven activities, with key contributors including greenhouse gas emissions, deforestation, and industrial practices."
Illustration: "Factory with smoke."

Slide 4: "Effects on Global Weather"
Script: "Climate change is causing more frequent and severe weather events like hurricanes, droughts, and heatwaves, impacting ecosystems and human societies worldwide."
Illustration: "Sun, cloud, and rain icon."

Slide 5: "Impact on Biodiversity"
Script: "The changing climate is leading to habitat destruction, species extinction, and altered ecosystems, posing threats to global biodiversity."
Illustration: "Tree and animal silhouette."

Slide 6: "Human Impact and Responses"
Script: "Human activities, notably fossil fuel combustion and land use changes, are primary drivers of climate change, prompting responses such as renewable energy development and conservation efforts."
Illustration: "Factory and tree."

Slide 7: "Policy and Regulation"
Script: "Governments and international organizations are formulating policies and regulations aimed at mitigating climate change, including international agreements like the Paris Accord."
Illustration: "Gavel and document."

Slide 8: "Technological Innovations"
Script: "Technological advancements play a crucial role in combating climate change, from renewable energy technologies to carbon capture and storage systems."
Illustration: "Wind turbine."

Slide 9: "Community and Cultural Impact"
Script: "Communities and cultures worldwide are adapting to climate change, incorporating sustainable practices and developing resilience against climate-related challenges."
Illustration: "Hands planting a seedling."

Slide 10: "Future Perspectives"
Script: "Future projections of climate change indicate continuing temperature rises, sea-level rise, and an increase in extreme weather events, necessitating urgent and comprehensive action."
Illustration: "Simple city skyline."

Answer presentation script on {topic}:
"""


structure_prompt_template = PromptTemplate(
    input_variables=["topic"],
    template=structure_prompt_template_text)


structure_chain = LLMChain(llm = llm, prompt=structure_prompt_template, verbose = True)

In [18]:
presentation = structure_chain.run("Beach")
print(presentation)



> Entering new LLMChain chain...
Prompt after formatting:

Based on these examples, generate a structured script for a presentation on "Beach" in about 10 slides. Ensure that each slide in the script:

Opens with a clear statement that introduces the distinct aspect of the topic.
Provides detailed points that give a comprehensive understanding of the slide's content, ideal for spoken narration.
Explores a different dimension of the topic to offer a well-rounded perspective.
Remember, the format should be consistent across all slides, with clear opening statements and detailed points, making the script easy to follow and effective for delivery.
Example presentation on Climate Change:

Example presentation on Climate Change:
Topic: Climate Change

Slide 1: "Introduction to Climate Change"
Script: "Climate change is a significant global phenomenon characterized by long-term shifts in temperature and weather patterns, primarily due to human activities."
Illustration: "Earth half green, h

In [19]:
import re

def extract_slide_details(input_string):
    # Define the regex pattern
    pattern = r'Slide\s(\d+):\s"([^"]+)"\s*\n\s*Script:\s"([^"]+)"\s*\n\s*Illustration:\s"([^"]+)"\s*\n\s*'

    # Find all matches
    matches = re.findall(pattern, input_string, re.DOTALL)

    # Transform the matches into an array of tuples (title, description)
    tmp = [(n, title, description, illustration) for n, title, description, illustration in matches]
    
    slide_details = []

    previous_n = 0

    for n, title, description, illustration in tmp:
        if int(n) > previous_n:
            slide_details.append((n, title, description, illustration))
            previous_n = int(n)
        else:
            break

    
    return slide_details

In [20]:
slide_details = extract_slide_details(presentation)

for slide in slide_details:
    n, title, description, illustration = slide
    print(f"Title: {title}")
    print(f"Script: {description}\n")
    print(f"Illustration: {illustration}\n")
    print("-----")

Title: Introduction to Beach
Script: Beach is a coastal landform characterized by sand, water, and other natural features, forming a popular tourist destination and recreational area.

Illustration: Sun, sand, and sea.

-----
Title: Types of Beaches
Script: Beaches can be categorized based on their geological features, such as sandy beaches, rocky beaches, and coral reef beaches.

Illustration: Sand, rock, and coral.

-----
Title: Beach Ecosystems
Script: Beaches are home to diverse ecosystems, including marine life, plants, and animals, which are vulnerable to climate change and other environmental threats.

Illustration: Sea turtle and coral.

-----
Title: Beach Management
Script: Beach management involves protecting and preserving the natural environment, managing tourism and recreation, and addressing issues like erosion and pollution.

Illustration: Beach umbrella and chair.

-----
Title: Beach Safety
Script: Beach safety is a critical concern, with risks including rip currents, s

In [13]:
!pip install gTTS

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [21]:
import requests

def call_stable_diffusion_text2img(prompt, fname):
    url = 'https://stablediffusionapi.com/api/v3/text2img'
    payload = {
        'prompt': prompt,
        'num_images': 1,  # Adjust as needed
        'key': ''# key 
        # Include other parameters as required by the API
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        data = response.json()
        if 'output' in data and data['output']:
            image_url = data['output'][0]
            image_response = requests.get(image_url)
            if image_response.status_code == 200:
                with open(fname, 'wb') as file:
                    file.write(image_response.content)
                return "Image saved successfully"
            else:
                print( "Failed to download the image")
        else:
            print("No image URL in response")
    else:
        print("Error")
        return response.status_code, response.text

In [22]:
from gtts import gTTS

def text_to_speech(text, filename):
    """
    Converts text to speech and saves it as an MP3 file.

    Args:
    text (str): The text to be converted to speech.
    filename (str): The name of the file to save the speech to.
    """
    # Creating a gTTS object
    tts = gTTS(text=text, lang='en')

    # Saving the converted audio to a file
    tts.save(filename)

In [25]:
for slide in slide_details:
    n, title, description, illustration = slide
    text_to_speech(description, n+".mp3")
    call_stable_diffusion_text2img(illustration, n+".jpg")

No image URL in response
Failed to download the image
No image URL in response
No image URL in response
No image URL in response


KeyboardInterrupt: 

In [52]:
!pip install moviepy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 61.2 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110730 sha256=fc9f729b010b67cc265e5afc565d0080186e700ecd513b923ba8b5bf8da0faf3
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [117]:
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips


image_clips = []
audio_clips = []
duration = 0

for slide in slide_details:
    n, title, description, illustration = slide 
    aud_clip = AudioFileClip(n+".mp3").set_fps(24)
    duration += aud_clip.duration
    img_clip = ImageClip(n+".jpg", duration=aud_clip.duration).set_fps(24)  # Convert duration to seconds    
    image_clips.append(img_clip)
    audio_clips.append(aud_clip)

# Concatenate the image clips and audio clips
video_clip = concatenate_videoclips(image_clips, method="compose").set_fps(24)
audio_clip = concatenate_audioclips(audio_clips).set_fps(24)

# Set the concatenated audio to the video clip
final_clip = video_clip.set_audio(audio_clip).set_duration(duration).set_fps(24).subclip(0,duration)
# Write the result to a file
final_clip.write_videofile("output_video.mp4", codec= "libx264", fps = 24)


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4


TypeError: must be real number, not NoneType

In [31]:
import subprocess

def create_video(slide_details):
    for slide in slide_details:
        n, title, description, illustration = slide

        # Convert image to a video clip with the same duration as the audio
        subprocess.call([
            'ffmpeg', '-loop', '1', '-i', f'{n}.jpg', '-i', f'{n}.mp3',
            '-c:v', 'libx264', '-tune', 'stillimage', '-c:a', 'aac', '-b:a', '192k',
            '-pix_fmt', 'yuv420p', '-shortest', f'{n}_temp.mp4'
        ])

    # Create a text file with all video files
    with open('file_list.txt', 'w') as file:
        for n, _, _, _ in slide_details:
            file.write(f"file '{n}_temp.mp4'\n")

    # Concatenate all video clips
    subprocess.call([
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', 'file_list.txt',
        '-c', 'copy', 'output_video.mp4'
    ])

    # Optional: Cleanup temporary files
    for n, _, _, _ in slide_details:
        subprocess.call(['rm', f'{n}_temp.mp4'])

create_video(slide_details)


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [77]:
!pip install ffmpeg --upgrade


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [114]:
!pip uninstall ffmpeg moviepy decorator -y
!pip install moviepy

Found existing installation: moviepy 1.0.3
Uninstalling moviepy-1.0.3:
  Successfully uninstalled moviepy-1.0.3
Found existing installation: decorator 4.0.2
Uninstalling decorator-4.0.2:
  Successfully uninstalled decorator-4.0.2
Defaulting to user installation because normal site-packages is not writeable
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [115]:
!pip install --upgrade decorator==4.0.2


Defaulting to user installation because normal site-packages is not writeable
  Using cached decorator-4.0.2-py2.py3-none-any.whl (8.5 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [98]:
!pip install pygame


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 84.1 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [118]:
!ffmpeg -version

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enabl

In [27]:
slide_details[2]

('3',
 'Beach Ecosystems',
 'Beaches are home to diverse ecosystems, including marine life, plants, and animals, which are vulnerable to climate change and other environmental threats.',
 'Sea turtle and coral.')

In [29]:
call_stable_diffusion_text2img("Sea turtle and coral.", "3.jpg")

'Image saved successfully'